In [2]:
# Make sure you have pymc3 installed on a conda distribution

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pymc3 as pm
import theano.tensor as tt
import theano.tensor.nnet as nnet
import theano.tensor.signal.conv
import theano
import arviz as az

In [ ]:
# For pip (3x as slow though):
# import aesara
# import pytensor
# import pytensor.tensor as tt  # it might not always find the conv sub function
# import aesara.tensor.nnet as nnet
# import aesara.tensor.signal.conv
# import arviz as az

# Importing data

In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np

series = pd.read_csv('data_2.csv', header=0, index_col=0, parse_dates=True)

# Time frame
start = '2022-11-7'
end = '2022-11-15'

series = series[start:end]

# Set all nan-values to zero
series['units_raw'] = series['units_raw'].fillna(0)
series['bolus'] = series['bolus'].fillna(0)
series['basal'] = series['basal'].fillna(0)

# Custom series
cgm_norm = series['cgm']  
insulin_norm = series['insulin']
basal_norm = series['basal']
bolus_norm = series['bolus']
units_raw = series['units_raw']
heart_rate = series['heart_rate']

# Creating a figure with 3 subplots (arranged vertically)
fig, axs = plt.subplots(3, 1, figsize=(15, 10), sharex=True)

# Plot the cgm_norm series on the first subplot
axs[0].plot(cgm_norm.index, cgm_norm, marker='x', linestyle='None', label='CGM', color='g') 
axs[0].fill_between(cgm_norm.index, 3, 10, color='lightgreen', zorder=-1, alpha=0.5)
axs[0].set_ylabel('Glucose (mmol)')
axs[0].set_title('Measured CGM values')

# Plot 2 - insulin_norm on the second subplot
axs[1].plot(bolus_norm.index, bolus_norm, label='Concentration', color='b', linewidth=2)
axs[1].plot(units_raw, label='Raw units', color='m', linewidth=2)
axs[1].set_ylabel('Insulin (U)')
axs[1].set_title('Bolus')

# Plot 3 - Basal rate
axs[2].plot(basal_norm.index, basal_norm, label='Concentration', color='b', linewidth=1)
axs[2].set_ylabel('Insulin (U)')
axs[2].set_title('Basal concentration')

# # Plot 4 - Units on the third subplot
# axs[3].plot(heart_rate.index, heart_rate, label='Heart Rate', color='c', linewidth=1)
# axs[3].set_ylabel('BPM')
# axs[3].set_xlabel('Time')
# axs[3].set_title('Recorded heart rate')

# Adding labels and legends to all subplots
for ax in axs:
    ax.set_xlim([pd.to_datetime(start), pd.to_datetime(end)])
    ax.legend(loc='upper left')   

# Day lines
for i in range(0, len(series.index)):
    if series.index[i].hour == 0 and series.index[i].minute == 0:
        axs[0].axvline(x=series.index[i], color='grey', linestyle='--')
        axs[1].axvline(x=series.index[i], color='grey', linestyle='--')
        axs[2].axvline(x=series.index[i], color='grey', linestyle='--')
        # axs[3].axvline(x=series.index[i], color='grey', linestyle='--')

# Show the plot
plt.rcParams.update({'font.size': 12})

plt.style.use('seaborn-whitegrid')
plt.rcParams['axes.facecolor'] = 'white'
plt.rcParams['axes.edgecolor'] = 'grey'
plt.rcParams['axes.linewidth'] = 2
plt.rcParams['grid.color'] = 'grey'
plt.rcParams['grid.linestyle'] = '--'
plt.rcParams['grid.linewidth'] = 0.5
plt.show()


# Down-sampling

In [ ]:
# Data resolution: how much we are downsampling our data. 
data_resolution = 30

# Downsample data
cgm_norm_15min = cgm_norm.resample(str(data_resolution) + 'min').mean()
insulin_norm_15min = insulin_norm.resample(str(data_resolution) + 'min').mean()
bolus_norm_15min = bolus_norm.resample(str(data_resolution) + 'min').mean()
basal_norm_15min = basal_norm.resample(str(data_resolution) + 'min').mean()

# Delete the last item, since that one is always nan because of the resampling.
cgm_norm_15min = cgm_norm_15min[:-1]
insulin_norm_15min = insulin_norm_15min[:-1]
bolus_norm_15min = bolus_norm_15min[:-1]
basal_norm_15min = basal_norm_15min[:-1]

# Single fit horizon

In [ ]:
# Define the horizon length
horizon_minutes = 1790
horizon = int(horizon_minutes / data_resolution)  # Adjust for your data resolution

# Extend the time series for the horizon
train_size = len(insulin_norm_15min) - horizon 
test_size = horizon

extended_size = train_size + horizon
integer_index = np.arange(extended_size) * data_resolution

# Define training and testing variables
cgm_train = cgm_norm_15min[:train_size]
cgm_test = cgm_norm_15min[train_size:]
bolus_train = bolus_norm_15min[:train_size]
bolus_test = bolus_norm_15min[train_size:]

print(cgm_test[-2], cgm_test[-1])

In [ ]:
# Plotting
plt.figure(figsize=(20, 7))
plt.plot(integer_index[:train_size], cgm_norm_15min[:train_size], label='Observed CGM', color='green', linewidth=2, marker='x')
plt.plot(integer_index[train_size:], cgm_norm_15min[train_size:], label='PH CGM', color='green', alpha=0.5, marker='x')

# Add vertical lines for train/test split and prediction horizon
plt.axvline(x=train_size * data_resolution, color='r', linestyle='--', label='Test start', zorder=10)
plt.axvline(x=(train_size + horizon - 1) * data_resolution, color='g', linestyle='--', label='Prediction Horizon', zorder=5)

# Fill range
plt.fill_between(integer_index, 3, 10, color='lightgreen', alpha=0.3)

# Plot a vertical line at every begin of a new day
for i in integer_index:
    if (i * data_resolution) % (1440 * data_resolution) == 0:
        plt.axvline(x=i, color='grey', linestyle='--')
        
plt.title('Prediction horizon')
plt.legend()
plt.xlabel('Minutes')
plt.ylabel('Insulin (U)')
plt.xlim(int(len(integer_index)) * data_resolution - 4500, min((int(len(integer_index)) * data_resolution + 20), (train_size + horizon) * data_resolution))
plt.show()

# Variables and activation functions

In [ ]:
# Calorie conversion to gram
calorie_per_gram_per_macro = {
    "carb": 4,
    "protein": 4,
    "fat": 9,
}
# Conversion rate:
conversion_ratios = {
    "carb":2.15,
    "protein":0.75,
    "fat":1.4
}

# Gram of carb to units of insulin ratio (g/U)
ic_ratio = 11.1
ip_ratio = 11.1
if_ratio = 11.1

# ------------------------------------------------------------------------------------------------

def normal_skew(units, mu=0.3, sigma=0.3, a=2, conversion_rate=conversion_ratios['carb'], scale=1):
    '''
    Additive effect of carbohydrates on blood glucose levels, modeled over time.
    
    Parameters
    ----------
    units :     NDArray[np.float32]
                number of units of insulin to counter effect, based on the g/U ratio for protein
    mu :        float
                mean of the normal distribution
    sigma :     float
                standard deviation of the normal distribution
    a :         float
                shape parameter of the Weibull distribution
    conversion_rate : float
                conversion rate of protein to units of insulin, used to see relative effect against other macronutrients 
    scale :     float
                scalar multiplicator 

    Returns
    ----------
    activation : NDArray[np.float32]
                additive effect of protein on blood glucose levels, modeled over time in range from 0 to 1000 minutes.
    '''
    x = np.linspace(0, 5, 1000)
    z = (x - mu) / sigma
    pdf = (1 / (sigma * tt.sqrt(2 * np.pi))) * tt.exp(-z ** 2 / 2)
    cdf = (1 + tt.erf(a * z / tt.sqrt(2)))
    activation = units * conversion_rate * pdf * cdf * scale
    return activation

def theano_carbohydrate_activation(units, alpha=2.3, beta=1.1, conversion_rate=conversion_ratios['carb']):
    '''
    Additive effect of carbohydrates on blood glucose levels, modeled over time.
    
    Parameters
    ----------
    units :     NDArray[np.float32]
                number of units of insulin to counter effect, based on the g/U ratio for carbs
    alpha :     float
                shape parameter of the Weibull distribution
    beta :      float
                scale parameter of the Weibull distribution
    conversion_rate : float
                conversion rate of carbs to units of insulin, used to see relative effect against other macronutrients 

    Returns
    ----------
    activation : NDArray[np.float32]
                additive effect of carbohydrates on blood glucose levels, modeled over time in range from 0 to 1000 minutes.
    '''
    x = np.linspace(0, 10, 1000)
    weibull = (alpha / beta) * (x / beta) ** (alpha - 1) * tt.exp(-(x / beta) ** alpha)
    activation = units * conversion_rate * weibull
    
    # Downside flush
    flush_scale = 0.3
    flush_units = units
    flush_mu = 1.4
    flush_sigma = 0.3
    flush_a = 1
    
    return activation - normal_skew(flush_units, mu=flush_mu, sigma=flush_sigma, scale=flush_scale, a=flush_a)


def theano_protein_activation(units, mu=1.1, sigma=0.5 , a=1, conversion_rate=conversion_ratios['protein']):
    '''
    Additive effect of protein on blood glucose levels, modeled over time.
    
    Parameters
    ----------
    units :     NDArray[np.float32]
                number of units of insulin to counter effect, based on the g/U ratio for protein
    mu :        float
                mean of the normal distribution
    sigma :     float
                standard deviation of the normal distribution
    a :         float
                shape parameter of the Weibull distribution
    conversion_rate : float
                conversion rate of protein to units of insulin, used to see relative effect against other macronutrients 

    Returns
    ----------
    activation : NDArray[np.float32]
                additive effect of protein on blood glucose levels, modeled over time in range from 0 to 1000 minutes.
    '''
    x = np.linspace(0, 5, 1000)
    z = (x - mu) / sigma
    pdf = (1 / (sigma * tt.sqrt(2 * np.pi))) * tt.exp(-z ** 2 / 2)
    cdf = (1 + tt.erf(a * z / tt.sqrt(2)))

    # Multiply every gamma function with the number of units
    activation = cdf * pdf * units * conversion_rate
    
    # Upside flush
    flush_scale = 0.045
    flush_units = units
    flush_mu = 0.7
    flush_sigma = 0.3
    flush_a = 2
    correct_up = normal_skew(flush_units, mu=flush_mu, sigma=flush_sigma, scale=flush_scale, a=flush_a)
    
    # Downside flush 2
    flush2_scale = 0.01
    flush2_units = units
    flush2_mu = 0.45
    flush2_sigma = 0.25
    flush2_a = 2
    correct_down = normal_skew(flush2_units, mu=flush2_mu, sigma=flush2_sigma, scale=flush2_scale, a=flush2_a)
    
    return activation + correct_up # - correct_down


def theano_fat_activation(units, mu=1.3, sigma=0.7 , a=1, conversion_rate=conversion_ratios['fat']):
    '''
    Additive effect of fat on blood glucose levels, modeled over time.
    
    Parameters
    ----------
    units :     NDArray[np.float32]
                number of units of insulin to counter effect, based on the g/U ratio for fat
    mu :        float
                mean of the normal distribution
    sigma :     float
                standard deviation of the normal distribution
    a :         float
                shape parameter of the Weibull distribution
    conversion_rate : float
                conversion rate of fat to units of insulin, used to see relative effect against other macronutrients 

    Returns
    ----------
    activation : NDArray[np.float32]
                additive effect of fat on blood glucose levels, modeled over time in range from 0 to 1000 minutes.
    '''
    x = np.linspace(0, 5, 1000)
    z = (x - mu) / sigma
    pdf = (1 / (sigma * tt.sqrt(2 * np.pi))) * tt.exp(-z ** 2 / 2)
    cdf = (1 + tt.erf(a * z / tt.sqrt(2)))
    
    fat_units = units * conversion_rate
    
    carb_scale = 0.08
    carb_units = units
    carb_mu = 0.2
    carb_sigma = 0.3
    carb_a = 2
    correct_down = normal_skew(carb_units, mu=carb_mu, sigma=carb_sigma, scale=carb_scale, a=carb_a)
    
    # Upside flush
    flush_scale = 0.04
    flush_units = units
    flush_mu = 0.9
    flush_sigma = 0.3
    flush_a = 2
    correct_up = normal_skew(flush_units, mu=flush_mu, sigma=flush_sigma, scale=flush_scale, a=flush_a)
    
    activation = fat_units * pdf * cdf 
    
    return activation - correct_down + correct_up

def theano_insulin_activation(units, sensitivity=1, alpha=2, rate=1.045):
    x = np.linspace(0, 16, 1000)  # Change the range to hours (0 to 8 hours)
    shifted_x = x - 12.28/60
    shifted_x[shifted_x <= 0] = 1e-8  # Replace non-positive values with a small positive value
    
    gamma = tt.exp(tt.gammaln(alpha) - tt.log(rate) * alpha - rate * shifted_x + (alpha - 1) * tt.log(shifted_x))
    
    # Multiply every gamma function with the number of units
    gamma = gamma * units * sensitivity

    return gamma

# The model

In [ ]:
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # #
#                        CONVOLUTION                      # 
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # #

def conv_activation(units, kernel):
    """
    Perform the convolution using theano.tensor.nnet.conv2d.

    Parameters
    ----------
    units : NDArray[np.float32]
            number of units of insulin to counter effect, based on the g/U ratio
    kernel : NDArray[np.float32]
             kernel representing the activation function

    Returns
    -------
    conv_result : NDArray[np.float32]
                  result of the convolution
    """
    # Reshape units and kernel tensors to meet the input requirements of conv2d
    units = units.dimshuffle('x', 'x', 0, 'x')
    kernel = kernel.dimshuffle('x', 'x', 0, 'x')

    # Perform convolution
    conv_result = pytensor.tensor.conv.conv2d(input=units, filters=kernel, border_mode='full')[0, 0, :, 0]

    return conv_result


def fourier_series(time, n_terms, amplitude, phase_shift, data_resolution, period=1440):
    time_adjusted = time * data_resolution  # make the time periodical over 1440 minutes
    result = 0
    for i in range(n_terms):
        result += amplitude[i] * (tt.sin(2 * np.pi * ((i+1) * (time_adjusted + phase_shift[i]) % period) / period) + 1) / 2
    return result



with pm.Model() as model:

    # --------------- MEAL TIMING --------------------

    # Meal availability fourier series
    n_meal_terms = 6
    meal_amplitude = pm.Uniform('meal_amplitude', lower=0, upper=0.2, shape=n_meal_terms)
    meal_phase_shift = pm.Uniform('meal_phase_shift', lower=0, upper=1440, shape=n_meal_terms)
    meal_prob = pm.Deterministic('mp', fourier_series(np.arange(extended_size), n_meal_terms, meal_amplitude, meal_phase_shift, data_resolution))
    mt = pm.Bernoulli('mt', p=meal_prob, shape=extended_size)

    
    # --------------- MEAL SIZE --------------------

    # Carb, protein, and fat calories per meal
    n_cpm_terms = 4  # Number of sinusoids for cpm
    cpm_amplitude = pm.Uniform('cpm_amplitude', lower=0, upper=100, shape=n_cpm_terms)  # assuming max meal size of 100 calories
    cpm_phase_shift = pm.Uniform('cpm_phase_shift', lower=0, upper=1440, shape=n_cpm_terms)
    cpm = pm.Deterministic('cpm', fourier_series(np.arange(extended_size), n_cpm_terms, cpm_amplitude, cpm_phase_shift, data_resolution))

    # Multiply the cpm values with mt
    cpm_binary = pm.Deterministic('cpmt', tt.switch(tt.eq(mt, 1), cpm, 0))
    
    
    # --------------- DIET COMPOSITION --------------------

    # Diet composition percentage logits
    logit_c = pm.Normal('logit_C', mu=0.442, sigma=0.075)
    logit_p = pm.Normal('logit_P', mu=0.147, sigma=0.04)
    logit_f = pm.Normal('logit_F', mu=0.365, sigma=0.064)

    # Apply softmax to obtain percentages summed to 1
    softmax_logits = tt.special.softmax([logit_c, logit_p, logit_f], axis=-1)

    # Percentages
    perc_c = pm.Deterministic('%C', softmax_logits[0])
    perc_p = pm.Deterministic('%P', softmax_logits[1])
    perc_f = pm.Deterministic('%F', softmax_logits[2])
    
    # Diet composition in grams per meal
    g_c = pm.Deterministic('g_C', cpm_binary * perc_c / calorie_per_gram_per_macro['carb'])
    g_p = pm.Deterministic('g_P', cpm_binary * perc_p / calorie_per_gram_per_macro['protein'])
    g_f = pm.Deterministic('g_F', cpm_binary * perc_f / calorie_per_gram_per_macro['fat'])

    # Insulin required for each macronutrient (g/U)
    ic = pm.Deterministic('IC', g_c / ic_ratio)
    ip = pm.Deterministic('IP', g_p / ip_ratio)
    if_ = pm.Deterministic('IF', g_f / if_ratio)


    # --------------- POSITIVE CONVOLUTION --------------------

    # Adjust the activation kernels to have a unit equal to 1
    carb_kernel = theano_carbohydrate_activation(1, conversion_rate=conversion_ratios['carb'])[::data_resolution]
    protein_kernel = theano_protein_activation(1, conversion_rate=conversion_ratios['protein'])[::data_resolution]
    fat_kernel = theano_fat_activation(1, conversion_rate=conversion_ratios['fat'])[::data_resolution]

    # Convert mt to float to be able to convolute
    mt = mt.astype('float64')

    # Convolution of meal availability with activation functions
    conv_carb = pm.Deterministic('conv_carb', conv_activation(ic, carb_kernel))
    conv_protein = pm.Deterministic('conv_protein', conv_activation(ip, protein_kernel))
    conv_fat = pm.Deterministic('conv_fat', conv_activation(if_, fat_kernel))

    # Total blood glucose level contribution from all macronutrients
    total_activation = pm.Deterministic('total_activation', conv_carb + conv_protein + conv_fat)
    
    # -------------------- NEGATIVE CONVOLUTION --------------------
    
    # Meal availability fourier series
    n_insulin_terms = 3
    insulin_amplitude = pm.Uniform('insulin_amplitude', lower=0, upper=0.3, shape=n_insulin_terms)
    insulin_phase_shift = pm.Uniform('insulin_phase_shift', lower=0, upper=1440, shape=n_insulin_terms)
    insulin_prob = pm.Deterministic('ip', fourier_series(np.arange(extended_size), n_insulin_terms, insulin_amplitude, insulin_phase_shift, data_resolution))
    it = pm.Bernoulli('it', p=insulin_prob, shape=extended_size)
    
    # Calories to units of insulin ratio
    calorie_insulin_ratio = pm.HalfNormal('calorie_insulin_ratio', tau=1/14)
    
    upm = pm.Deterministic('upm', cpm / calorie_insulin_ratio)
    
    # Multiply the cpm values with mt
    insulin_binary = pm.Deterministic('upmt', tt.switch(tt.eq(it, 1), upm, 0))
    
    # Create kernel
    insulin_kernel = theano_insulin_activation(1)[::data_resolution]
    bolus_insulin_conv = pm.Deterministic('conv_insulin', conv_activation(insulin_binary, insulin_kernel))
    
    insulin_sd = pm.Exponential('insulin_sigma', lam=1/0.5)
    bolus_insulin = pm.Normal('bolus_insulin', mu=bolus_insulin_conv[:train_size], sigma=insulin_sd, observed=bolus_train)

    # --------------- PASSIVE COMPONENT --------------------

    # Basal glucose production
    n_basal_terms = 1
    basal_amplitude = pm.Uniform('basal_amplitude', lower=0, upper=1, shape=n_basal_terms)
    basal_phase_shift = pm.Uniform('basal_phase_shift', lower=0, upper=1440, shape=n_basal_terms)

    basal_offset = pm.Normal('basal_baseline', mu=3, sigma=1)
    basal_active = pm.Deterministic('basal_active', fourier_series(np.arange(extended_size), n_basal_terms, basal_amplitude, basal_phase_shift, data_resolution) + basal_offset)    
    total_basal = pm.ConstantData('total_basal', basal_norm_15min[:extended_size])

    # --------------- COMBINING VARIABLES ---------------

    # Calculate component effects
    total_active = pm.Deterministic('total_active', total_activation[:extended_size] - bolus_insulin_conv[:extended_size])  # Total active component
    total_passive = pm.Deterministic('total_passive', basal_active - total_basal)  # Total passive component

    # Theoretical CGM readings
    total_cgm = pm.Deterministic('cgm_est', total_active[:train_size] + total_passive[:train_size])

    # Define the likelihood of the model
    sigma = pm.Exponential('sigma', lam=1/0.5)
    likelihood = pm.Normal('likelihood', mu=total_cgm, sigma=sigma, observed=cgm_train)

    # --------------- SAMPLING ---------------

    # Run the MCMC sampling
    step = pm.NUTS(target_accept=0.95)
    trace = pm.sample(draws=2000, tune=1000, chains=4, cores=4, step=step, return_inferencedata=True)
    

## Plotting macronutrient concentration

In [ ]:
plt.figure(figsize=(20, 7))

# Define the macronutrients
macronutrients = ['conv_carb', 'conv_protein', 'conv_fat']
names = ['Carbs', 'Protein', 'Fats']
colors = ['purple', 'green', 'orange']

# For each macronutrient
for i, macronutrient in enumerate(macronutrients):
    # Calculate the median and percentiles
    median = np.median(trace.posterior[macronutrient].values, axis=(0, 1))[:extended_size]
    low_50, high_50 = np.percentile(trace.posterior[macronutrient].values, [25, 75], axis=(0, 1))[:extended_size]
    low_75, high_75 = np.percentile(trace.posterior[macronutrient].values, [12.5, 87.5], axis=(0, 1))[:extended_size]
    low_95, high_95 = np.percentile(trace.posterior[macronutrient].values, [2.5, 97.5], axis=(0, 1))[:extended_size]
    
    # Plot the median and percentiles
    plt.fill_between(integer_index, low_50[:extended_size], high_50[:extended_size], color=colors[i], alpha=0.4)
    plt.fill_between(integer_index, low_75[:extended_size], high_75[:extended_size], color=colors[i], alpha=0.3)
    plt.fill_between(integer_index, low_95[:extended_size], high_95[:extended_size], color=colors[i], alpha=0.2)
    plt.plot(integer_index, median, linewidth=5, label=names[i], color=colors[i])
    
# Lines
plt.axhline(y=0, color='black', linewidth=1)
plt.axvline(x=train_size * data_resolution, color='r', linestyle='-', label='Train/Test Split')
# plt.axvline(x=(train_size + horizon) * data_resolution, color='g', linestyle='-', label='Prediction Horizon')

plt.legend(loc='upper right')
plt.xlim(int(len(integer_index)) * data_resolution - 4500, int(len(integer_index)) * data_resolution + 20)
plt.title('Convoluted macros')
plt.xlabel('Time (minutes)')
plt.ylabel('Glucose exursion (mmol/L)')

# Plot a vertical line at every begin of a new day
for i in integer_index:
    if (i * data_resolution) % (1440 * data_resolution) == 0:
        plt.axvline(x=i, color='grey', linestyle='--')
        
plt.show()

## Plotting insulin concentration

In [ ]:
insulin_sigma_samples = trace.posterior['insulin_sigma'].values
conv_insulin_samples = trace.posterior['conv_insulin'].values

# Calculate the mean of sigma_samples along the first two dimensions
insulin_mean_sigma_samples = np.mean(insulin_sigma_samples, axis=(0, 1))

# Generate noise with shape matching total_active (and total_passive)
noise = np.random.normal(loc=0, scale=insulin_mean_sigma_samples, size=conv_insulin_samples.shape)

# Extract basal_active values
bolus_insulin_values = conv_insulin_samples + noise

# Calculate the median and percentiles
bolus_insulin_active_median = np.median(bolus_insulin_values, axis=(0, 1))[:extended_size]
bolus_insulin_active_low_50, bolus_insulin_active_high_50 = np.percentile(bolus_insulin_values, [25, 75], axis=(0, 1))
bolus_insulin_active_low_75, bolus_insulin_active_high_75 = np.percentile(bolus_insulin_values, [12.5, 87.5], axis=(0, 1))
bolus_insulin_active_low_95, bolus_insulin_active_high_95 = np.percentile(bolus_insulin_values, [2.5, 97.5], axis=(0, 1))

# Calculate the time index
integer_index = np.arange(extended_size) * data_resolution

# Create subplots
plt.figure(figsize=(20, 7))

# Plot the median and percentiles
plt.fill_between(integer_index, bolus_insulin_active_low_50[:extended_size], bolus_insulin_active_high_50[:extended_size], color='red', alpha=0.6)
plt.fill_between(integer_index, bolus_insulin_active_low_75[:extended_size], bolus_insulin_active_high_75[:extended_size], color='red', alpha=0.4)
plt.fill_between(integer_index, bolus_insulin_active_low_95[:extended_size], bolus_insulin_active_high_95[:extended_size], color='red', alpha=0.2)

# Plot the median line
plt.plot(integer_index, bolus_insulin_active_median, linewidth=5, label='Posterior median', color='black')
plt.plot(integer_index[:train_size], bolus_norm_15min[:train_size], linewidth=5, label='Observed', color='blue')
plt.plot(integer_index[train_size:], bolus_norm_15min[train_size:], linewidth=5, color='blue', alpha=0.5)
plt.axvline(x=train_size * data_resolution, color='r', linestyle='-', label='Train/Test Split')
plt.axvline(x=(train_size + horizon) * data_resolution, color='g', linestyle='-')

# Add legend to the plot
plt.legend(loc='upper left')
plt.xlim(int(len(integer_index)) * data_resolution - 4500, int(len(integer_index)) * data_resolution + 20)
plt.title('Insulin')
plt.xlabel('Time (minutes)')
plt.ylabel('Insulin (U)')

## Plotting final estimated CGM

In [ ]:
# Extracting values from the trace

plt.rcParams.update({'font.size': 18})

sigma_samples = trace.posterior['sigma'].values
total_active = trace.posterior["total_active"].values
total_passive = trace.posterior["total_passive"].values

# Calculate the mean of sigma_samples along the first two dimensions
mean_sigma_samples = np.mean(sigma_samples, axis=(0, 1))

# Generate noise with shape matching total_active (and total_passive)
noise = np.random.normal(loc=0, scale=mean_sigma_samples, size=total_active.shape)

# Add all components together
net = total_active + total_passive + noise

net_median = np.median(net, axis=(0, 1))
low_50, high_50 = np.percentile(net, [25, 75], axis=0)
low_75, high_75 = np.percentile(net, [12.5, 87.5], axis=0)
low_95, high_95 = np.percentile(net, [2.5, 97.5], axis=0)

# Calculate the time index
integer_index = np.arange(extended_size) * data_resolution

# Create subplots
plt.figure(figsize=(20, 7))

# # Subplot for glucose
plt.fill_between(integer_index, np.mean(low_50, axis=0), np.mean(high_50, axis=0), color='red', alpha=0.6)
plt.fill_between(integer_index, np.mean(low_75, axis=0), np.mean(high_75, axis=0), color='red', alpha=0.4)
plt.fill_between(integer_index, np.mean(low_95, axis=0), np.mean(high_95, axis=0), color='red', alpha=0.2)
plt.plot(integer_index[train_size:], cgm_norm_15min[train_size:extended_size], label='True CGM', linewidth=5, color='green', alpha=0.5, zorder=15, marker='x')
plt.plot(integer_index[:train_size], cgm_train, label='Observed CGM', linewidth=5, color='green')
plt.plot(integer_index, net_median, label='Posterior median', color='black', linewidth=5)

# Lines
plt.fill_between(integer_index, 3, 10, color='lightgreen', alpha=0.4, zorder=-1)
plt.axvline(x=train_size * data_resolution, color='r', linestyle='-', label='Train/Test Split', linewidth=4)
plt.axvline(x=(train_size + horizon) * data_resolution, color='g', linestyle='-')
# axs[0].axhline(y=3, color='red', linestyle='--')
# axs[0].axhline(y=10, color='red', linestyle='--')

plt.legend()
plt.xlim(int(len(integer_index)) * data_resolution - 4500, int(len(integer_index)) * data_resolution + 20)
# plt.xlim(int(len(integer_index)) * data_resolution - 6000, train_size * data_resolution)
plt.ylim(2, 14)
plt.title('Fitted and predicted glucose')
plt.xlabel('Time (minutes)')
plt.ylabel('mmol/L')

for i in integer_index:
    if (i * data_resolution) % (1440 * data_resolution) == 0:
        plt.axvline(x=i, color='grey', linestyle='--')
        


## Passive compartment

In [ ]:
# Create subplots
fig, axs = plt.subplots(2, 1, figsize=(10, 10))

# Extract basal_active values
basal_active_values = trace.posterior['basal_active'].values[:, :, :extended_size]

# Calculate the median and percentiles
basal_active_median = np.median(basal_active_values, axis=(0, 1))
basal_active_low_50, basal_active_high_50 = np.percentile(basal_active_values, [25, 75], axis=(0, 1))
basal_active_low_75, basal_active_high_75 = np.percentile(basal_active_values, [12.5, 87.5], axis=(0, 1))
basal_active_low_95, basal_active_high_95 = np.percentile(basal_active_values, [2.5, 97.5], axis=(0, 1))

# Plot the median and percentiles
axs[0].fill_between(integer_index, basal_active_low_50, basal_active_high_50, color='red', alpha=0.6)
axs[0].fill_between(integer_index, basal_active_low_75, basal_active_high_75, color='red', alpha=0.4)
axs[0].fill_between(integer_index, basal_active_low_95, basal_active_high_95, color='red', alpha=0.2)

# Plot the median line
axs[0].plot(integer_index, basal_active_median, linewidth=5, label='Posterior median', color='black')
# axs[0].axvline(x=train_size * data_resolution, color='r', linestyle='-', label='Train/Test Split')
# axs[0].axvline(x=(train_size + horizon) * data_resolution, color='g', linestyle='-', label='Prediction Horizon')

# Add legend to the plot
axs[0].legend()
axs[0].set_xlim(0, 1440)
axs[0].set_title('Endogenous Glucose Production')
axs[0].set_xlabel('Time (minutes)')
axs[0].set_ylabel('Glucose exursion (mmol/L)')

# --------------------------------------------
# Calculate the median and percentiles
basal_active_median = np.median(basal_active_values, axis=(0, 1)) - np.median(trace.posterior['basal_baseline'].values, axis=(0, 1))
basal_active_low_50, basal_active_high_50 = np.percentile(basal_active_values, [25, 75], axis=(0, 1)) - np.median(trace.posterior['basal_baseline'].values, axis=(0, 1))
basal_active_low_75, basal_active_high_75 = np.percentile(basal_active_values, [12.5, 87.5], axis=(0, 1)) - np.median(trace.posterior['basal_baseline'].values, axis=(0, 1))
basal_active_low_95, basal_active_high_95 = np.percentile(basal_active_values, [2.5, 97.5], axis=(0, 1)) - np.median(trace.posterior['basal_baseline'].values, axis=(0, 1))

# Plot the median and percentiles
axs[1].fill_between(integer_index, basal_active_low_50, basal_active_high_50, color='red', alpha=0.6)
axs[1].fill_between(integer_index, basal_active_low_75, basal_active_high_75, color='red', alpha=0.4)
axs[1].fill_between(integer_index, basal_active_low_95, basal_active_high_95, color='red', alpha=0.2)

# Plot the median line
axs[1].plot(integer_index, basal_active_median, linewidth=5, label='Posterior median', color='black')
# axs[1].axvline(x=train_size * data_resolution, color='r', linestyle='-', label='Train/Test Split')
# axs[1].axvline(x=(train_size + horizon) * data_resolution, color='g', linestyle='-', label='Prediction Horizon')

# Add legend to the plot
axs[1].legend(loc='lower center')
axs[1].set_xlim(0, 1440)
axs[1].set_title('Circadian addition')
axs[1].set_xlabel('Time (minutes)')
axs[1].set_ylabel('Glucose exursion (mmol/L)')

plt.tight_layout()
plt.show()

## Basal baseline

In [ ]:
import seaborn as sns

sns.kdeplot(np.median(trace.posterior['basal_baseline'].values, axis=0), label='Basal baseline', color='purple', linewidth=2)

plt.title('Endogenous Glucose Baseline')
plt.xlabel('mmol/L')
plt.show()

## Calories per unit

In [ ]:
import seaborn as sns

sns.kdeplot(np.median(trace.posterior['calorie_insulin_ratio'].values, axis=0), label='Basal baseline', color='purple', linewidth=2)

plt.title('Calories to insulin ratio')
plt.xlabel('cal/U')
plt.show()

## Macronutrient composition

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import norm

# Define priors
prior_means = [0.442, 0.147, 0.365]  # Mean values for carbs, protein, fat
prior_stds = [0.075, 0.04, 0.064]  # Standard deviations for carbs, protein, fat
colors = ['purple', 'green', 'orange']  # Colors for carbs, protein, fat
labels = ['Carb', 'Protein', 'Fat']  # Labels for carbs, protein, fat

# Define range of values
x = np.linspace(0, 1, 1000)

# Initialize figure
plt.figure(figsize=(10, 5))
plt.title('Posterior and Prior Macro Composition Percentages')

# Loop through macros
for i in range(3):
    # Draw prior
    prior = norm.pdf(x, prior_means[i], prior_stds[i])
    plt.fill_between(x, prior, color=colors[i], alpha=0.2)

    # Draw posterior
    posterior_samples = np.median(trace.posterior[f'logit_{labels[i][0]}'].values, axis=(0))
    sns.kdeplot(posterior_samples, label=labels[i], color=colors[i], linewidth=0, fill=True, alpha=0.7)
#     plt.hist(posterior_samples, bins=35, alpha=0.5, label=None, density=True, color=colors[i], edgecolor=None)

# Customize plot
plt.xlabel('Percentage')
plt.xlim(0, 1)
plt.legend()
plt.show()


## Plotting a Fourier series

In [ ]:
plt.rcParams.update({'font.size': 22})

def fourier_series_np(time, n_terms, amplitude, phase_shift, data_resolution, period=1440):
    time_adjusted = time * data_resolution  # make the time periodical over 1440 minutes
    result = 0
    for i in range(n_terms):
        result += amplitude[i] * (np.sin(2 * np.pi * ((i+1) * (time_adjusted + phase_shift[i]) % period) / period) + 1) / 2
    return result


def plot_fourier_series(trace, variable_name, n_terms):
    fig, axs = plt.subplots(1, 2, figsize=(15, 7))

    # Extract amplitude and phase_shift values
    amplitude_values = trace.posterior[variable_name + '_amplitude'].values
    phase_shift_values = trace.posterior[variable_name + '_phase_shift'].values

    # Prepare an empty array to hold the Fourier series results for each term
    fourier_series_results = np.empty((amplitude_values.shape[0], amplitude_values.shape[1], extended_size))
    fourier_series_results_terms = np.empty((n_terms, amplitude_values.shape[0], amplitude_values.shape[1], extended_size))
    time = np.arange(extended_size)

    # Calculate the Fourier series for each sample and each set of amplitude and phase shift values
    for i in range(amplitude_values.shape[0]):
        for j in range(amplitude_values.shape[1]):
            fourier_series_results[i, j, :] = fourier_series_np(time, n_terms, amplitude_values[i, j, :], phase_shift_values[i, j, :], data_resolution)
            
    # Calculate the Fourier series for each term and each set of amplitude and phase shift values
    for term in range(n_terms):
        for i in range(amplitude_values.shape[0]):
            for j in range(amplitude_values.shape[1]):
                amplitude_term = np.zeros(n_terms)
                amplitude_term[term] = amplitude_values[i, j, term]
                fourier_series_results_terms[term, i, j, :] = fourier_series_np(time, n_terms, amplitude_term, phase_shift_values[i, j, :], data_resolution)

    # Calculate the median and percentiles at each time step
    fourier_series_median = np.median(fourier_series_results, axis=(0, 1))
    fourier_series_low_50, fourier_series_high_50 = np.percentile(fourier_series_results, [25, 75], axis=(0, 1))
    fourier_series_low_75, fourier_series_high_75 = np.percentile(fourier_series_results, [12.5, 87.5], axis=(0, 1))
    fourier_series_low_95, fourier_series_high_95 = np.percentile(fourier_series_results, [2.5, 97.5], axis=(0, 1))

    # Calculate the median and percentiles at each time step for each term
    fourier_series_terms_median = np.median(fourier_series_results_terms, axis=(1, 2))
    fourier_series_terms_low_50, fourier_series_terms_high_50 = np.percentile(fourier_series_results_terms, [25, 75], axis=(1, 2))
    fourier_series_terms_low_75, fourier_series_terms_high_75 = np.percentile(fourier_series_results_terms, [12.5, 87.5], axis=(1, 2))
    fourier_series_terms_low_95, fourier_series_terms_high_95 = np.percentile(fourier_series_results_terms, [2.5, 97.5], axis=(1, 2))

    # Subplot for the full Fourier series
    axs[0].plot(time * data_resolution, fourier_series_median, color='black', label='Median (full series)')
    axs[0].fill_between(time * data_resolution, fourier_series_low_50, fourier_series_high_50, color='gray', alpha=0.4, label='50% CI (full series)')
    axs[0].set_title('Meal probability (mp)')
    axs[0].set_xlabel('Minutes in a day')
    axs[0].set_ylabel('Probability')
    axs[0].set_xlim(0, 1440)
    axs[0].legend()

    # Subplot for the underlying terms
    colors = ['blue', 'red', 'green', 'purple', 'orange', 'gray']  # Define colors for each term
    i = 12
    for term in range(n_terms):
        axs[1].plot(time * data_resolution, fourier_series_terms_median[term, :], color=colors[term], label=f'Median (term {term+1})', linewidth=i)
        i -= 2
#         axs[1].fill_between(time * data_resolution, fourier_series_terms_low_50[term, :], fourier_series_terms_high_50[term, :], color=colors[term], alpha=0.2, label=f'50% CI (term {term+1})')
    axs[1].set_title('Underlying terms')
    axs[1].set_xlabel('Minutes in a day')
    axs[1].set_ylabel('Probability')
    axs[1].set_xlim(0, 1440)
    axs[1].legend()

    plt.tight_layout()
    plt.show()


plot_fourier_series(trace, 'meal', 6)


# Utility functions 

In [ ]:
def get_plotting_variables(trace):
    # Define dictionaries to hold results
    plot_dict_glucose = {}
    
    # Extracting values from the trace
    total_active = trace.posterior["total_active"].values
    total_passive = trace.posterior["total_passive"].values

    # Calculate the mean of sigma_samples along the first two dimensions
    sigma_samples = trace.posterior['sigma'].values
    mean_sigma_samples = np.mean(sigma_samples, axis=(0, 1))

    # Generate noise with shape matching total_active (and total_passive)
    noise = np.random.normal(loc=0, scale=mean_sigma_samples, size=total_active.shape)

    # Add all components together
    net = total_active + total_passive + noise

    # Calculating percentiles
    plot_dict_glucose['median'] = np.median(net, axis=(0, 1))
    plot_dict_glucose['low_50'], plot_dict_glucose['high_50'] = np.percentile(net, [25, 75], axis=0)
    plot_dict_glucose['low_75'], plot_dict_glucose['high_75'] = np.percentile(net, [12.5, 87.5], axis=0)
    plot_dict_glucose['low_95'], plot_dict_glucose['high_95'] = np.percentile(net, [2.5, 97.5], axis=0)

    return plot_dict_glucose


In [ ]:
def fit_and_return_trace(cgm_train, bolus_train, basal_norm_15min, train_size, extended_size):

    # # # # # # # # # # # # # # # # # # # # # # # # # # # # # #
    #                        CONVOLUTION                      # 
    # # # # # # # # # # # # # # # # # # # # # # # # # # # # # #

    def conv_activation(units, kernel):
        """
        Perform the convolution using theano.tensor.nnet.conv2d.

        Parameters
        ----------
        units : NDArray[np.float32]
                number of units of insulin to counter effect, based on the g/U ratio
        kernel : NDArray[np.float32]
                 kernel representing the activation function

        Returns
        -------
        conv_result : NDArray[np.float32]
                      result of the convolution
        """
        # Reshape units and kernel tensors to meet the input requirements of conv2d
        units = units.dimshuffle('x', 'x', 0, 'x')
        kernel = kernel.dimshuffle('x', 'x', 0, 'x')

        # Perform convolution
        conv_result = nnet.conv2d(input=units, filters=kernel, border_mode='full')[0, 0, :, 0]
        
#         For pip:
#         conv_result = pytensor.tensor.conv.conv2d(input=units, filters=kernel, border_mode='full')[0, 0, :, 0]

        return conv_result


    def fourier_series(time, n_terms, amplitude, phase_shift, data_resolution, period=1440):
        time_adjusted = time * data_resolution  # make the time periodical over 1440 minutes
        result = 0
        for i in range(n_terms):
            result += amplitude[i] * (tt.sin(2 * np.pi * ((i+1) * (time_adjusted + phase_shift[i]) % period) / period) + 1) / 2
        return result



    with pm.Model() as model:

        # --------------- MEAL TIMING --------------------

        # Meal availability fourier series
        n_meal_terms = 1
        meal_amplitude = pm.Uniform('meal_amplitude', lower=0, upper=1, shape=n_meal_terms)
        meal_phase_shift = pm.Uniform('meal_phase_shift', lower=0, upper=1440, shape=n_meal_terms)
        meal_prob = pm.Deterministic('mp', fourier_series(np.arange(extended_size), n_meal_terms, meal_amplitude, meal_phase_shift, data_resolution))
        mt = pm.Bernoulli('mt', p=meal_prob, shape=extended_size)


        # --------------- MEAL SIZE --------------------

        # Carb, protein, and fat grams per meal
        n_cpm_terms = 4  # Number of sinusoids for cpm
        cpm_amplitude = pm.Uniform('cpm_amplitude', lower=0, upper=100, shape=n_cpm_terms)  # assuming max meal size of 100 calories
        cpm_phase_shift = pm.Uniform('cpm_phase_shift', lower=0, upper=1440, shape=n_cpm_terms)
        cpm = pm.Deterministic('cpm', fourier_series(np.arange(extended_size), n_cpm_terms, cpm_amplitude, cpm_phase_shift, data_resolution))

        # Multiply the cpm values with mt
        cpm_binary = pm.Deterministic('cpmt', tt.switch(tt.eq(mt, 1), cpm, 0))


        # --------------- DIET COMPOSITION --------------------

        # Diet composition percentage logits
        logit_c = pm.Normal('logit_C', mu=0.442, sd=0.075)
        logit_p = pm.Normal('logit_P', mu=0.147, sd=0.04)
        logit_f = pm.Normal('logit_F', mu=0.365, sd=0.064)

        # Apply softmax to obtain percentages summed to 1
        softmax_logits = tt.nnet.softmax([logit_c, logit_p, logit_f])

        # Percentages
        perc_c = pm.Deterministic('%C', softmax_logits[0, 0])
        perc_p = pm.Deterministic('%P', softmax_logits[0, 1])
        perc_f = pm.Deterministic('%F', softmax_logits[0, 2])

        # Diet composition in grams per meal
        g_c = pm.Deterministic('g_C', cpm_binary * perc_c / calorie_per_gram_per_macro['carb'])
        g_p = pm.Deterministic('g_P', cpm_binary * perc_p / calorie_per_gram_per_macro['protein'])
        g_f = pm.Deterministic('g_F', cpm_binary * perc_f / calorie_per_gram_per_macro['fat'])

        # Insulin required for each macronutrient (g/U)
        ic = pm.Deterministic('IC', g_c / ic_ratio)
        ip = pm.Deterministic('IP', g_p / ip_ratio)
        if_ = pm.Deterministic('IF', g_f / if_ratio)


        # --------------- POSITIVE CONVOLUTION --------------------

        # Adjust the activation kernels to have a unit equal to 1
        carb_kernel = theano_carbohydrate_activation(1, conversion_rate=conversion_ratios['carb'])[::data_resolution]
        protein_kernel = theano_protein_activation(1, conversion_rate=conversion_ratios['protein'])[::data_resolution]
        fat_kernel = theano_fat_activation(1, conversion_rate=conversion_ratios['fat'])[::data_resolution]

        # Convert mt to float to be able to convolute
        mt = mt.astype('float64')

        # Convolution of meal availability with activation functions
        conv_carb = pm.Deterministic('conv_carb', conv_activation(ic, carb_kernel))
        conv_protein = pm.Deterministic('conv_protein', conv_activation(ip, protein_kernel))
        conv_fat = pm.Deterministic('conv_fat', conv_activation(if_, fat_kernel))

        # Total blood glucose level contribution from all macronutrients
        total_activation = pm.Deterministic('total_activation', conv_carb + conv_protein + conv_fat)

        # -------------------- NEGATIVE CONVOLUTION --------------------

        # Meal availability fourier series
        n_insulin_terms = 3
        insulin_amplitude = pm.Uniform('insulin_amplitude', lower=0, upper=0.3, shape=n_insulin_terms)
        insulin_phase_shift = pm.Uniform('insulin_phase_shift', lower=0, upper=1440, shape=n_insulin_terms)
        insulin_prob = pm.Deterministic('ip', fourier_series(np.arange(extended_size), n_insulin_terms, insulin_amplitude, insulin_phase_shift, data_resolution))
        it = pm.Bernoulli('it', p=insulin_prob, shape=extended_size)

        # Calories to units of insulin ratio
        calorie_insulin_ratio = pm.HalfNormal('calorie_insulin_ratio', tau=1/14)

        upm = pm.Deterministic('upm', cpm / calorie_insulin_ratio)

        # Multiply the cpm values with mt
        insulin_binary = pm.Deterministic('upmt', tt.switch(tt.eq(it, 1), upm, 0))

        # Create kernel
        insulin_kernel = theano_insulin_activation(1)[::data_resolution]
        bolus_insulin_conv = pm.Deterministic('conv_insulin', conv_activation(insulin_binary, insulin_kernel))

        insulin_sd = pm.Exponential('insulin_sigma', lam=1/0.5)
        bolus_insulin = pm.Normal('bolus_insulin', mu=bolus_insulin_conv[:train_size], sd=insulin_sd, observed=bolus_train)

        # --------------- PASSIVE COMPONENT --------------------

        # Basal glucose production
        n_basal_terms = 1
        basal_amplitude = pm.Uniform('basal_amplitude', lower=0, upper=1, shape=n_basal_terms)
        basal_phase_shift = pm.Uniform('basal_phase_shift', lower=0, upper=1440, shape=n_basal_terms)

        basal_offset = pm.Normal('basal_baseline', mu=3, sd=1)
        basal_active = pm.Deterministic('basal_active', fourier_series(np.arange(extended_size), n_basal_terms, basal_amplitude, basal_phase_shift, data_resolution) + basal_offset)    
        total_basal = pm.Data('total_basal', basal_norm_15min[:extended_size])

        # --------------- COMBINING VARIABLES ---------------

        # Calculate component effects
        total_active = pm.Deterministic('total_active', total_activation[:extended_size] - bolus_insulin_conv[:extended_size])  # Total active component
        total_passive = pm.Deterministic('total_passive', basal_active - total_basal)  # Total passive component

        # Theoretical CGM readings
        total_cgm = pm.Deterministic('cgm_est', total_active[:train_size] + total_passive[:train_size])

        # Define the likelihood of the model
        sigma = pm.Exponential('sigma', lam=1/0.5)
        likelihood = pm.Normal('likelihood', mu=total_cgm, sd=sigma, observed=cgm_train)

        # --------------- SAMPLING ---------------

        # Run the MCMC sampling
        step = pm.NUTS(target_accept=0.95)
        trace = pm.sample(draws=2000, tune=1000, chains=4, cores=4, step=step, return_inferencedata=True)
    
    return trace


In [ ]:
from math import sqrt
from sklearn.metrics import mean_squared_error
import numpy as np

# Define hypoglycemia threshold
threshold = 70 / 18.0182

# Define function to generate binary hypoglycemia series
def generate_hypoglycemia_series(data, threshold):
    data = np.array(data)  # Convert data to a numpy array
    hypoglycemia_series = (data < threshold).astype(int)
    return hypoglycemia_series

def evaluate(test_data, pred_array):
    # ---- RMSE --------
    mse = mean_squared_error(test_data, pred_array)
    rmse = sqrt(mse)
    print('RMSE: {}'.format(round(rmse * 18.0182, 2)))  # Multiply by 18.0182 to convert from mmol/L to mg/dL

    # ---- MCC --------
    # Generate binary actual and predicted hypoglycemia series
    actual_hypoglycemia_series = generate_hypoglycemia_series(test_data, threshold)
    predicted_hypoglycemia_series = generate_hypoglycemia_series(pred_array, threshold)

    # Compute TP, FP, FN, and TN
    TP = np.sum((predicted_hypoglycemia_series == 1) & (actual_hypoglycemia_series == 1))
    FP = np.sum((predicted_hypoglycemia_series == 1) & (actual_hypoglycemia_series == 0))
    FN = np.sum((predicted_hypoglycemia_series == 0) & (actual_hypoglycemia_series == 1))
    TN = np.sum((predicted_hypoglycemia_series == 0) & (actual_hypoglycemia_series == 0))

    # Compute MCC
    mcc = (TP * TN - FP * FN) / (np.sqrt((TP + FP) * (TP + FN) * (TN + FP) * (TN + FN)) + 1e-10)

    return round(rmse * 18.0182, 2), mcc

# Iterative results

In [ ]:
horizon_minutes =  1790

# Define result storing arrays
glucose_median_pred = [[] for i in range(horizon_minutes // data_resolution)]
glucose_low_50_pred, glucose_high_50_pred = [[] for i in range(horizon_minutes // data_resolution)], [[] for i in range(horizon_minutes // data_resolution)]
glucose_low_75_pred, glucose_high_75_pred = [[] for i in range(horizon_minutes // data_resolution)], [[] for i in range(horizon_minutes // data_resolution)]
glucose_low_95_pred, glucose_high_95_pred = [[] for i in range(horizon_minutes // data_resolution)], [[] for i in range(horizon_minutes // data_resolution)]

# # These will be calculated afterwards
# rmse_median, rmse_lower, rmse_higher = [[[] for _ in cgm_test.values] for _ in look_ahead_intervals], [[[] for _ in cgm_test.values] for _ in look_ahead_intervals], [[[] for _ in cgm_test.values] for _ in look_ahead_intervals]
# mcc_median, mcc_lower, mcc_higher = [[[] for _ in cgm_test.values] for _ in look_ahead_intervals], [[[] for _ in cgm_test.values] for _ in look_ahead_intervals], [[[] for _ in cgm_test.values] for _ in look_ahead_intervals]


### Intermediate results

In [ ]:
i = 0
while horizon_minutes > 0:
    print("Running length", horizon_minutes)

   # ----------------- CREATE VARIABLES -----------------
    # Define the horizon length
    horizon = int(horizon_minutes / data_resolution)  # Adjust for data resolution

    # Extend the time series for the horizon
    train_size = len(insulin_norm_15min) - horizon
    test_size = horizon
    extended_size = train_size + horizon

    # Define training and testing variables
    cgm_train = cgm_norm_15min[:train_size]
    cgm_test = cgm_norm_15min[train_size:]
    bolus_train = bolus_norm_15min[:train_size]
    bolus_test = bolus_norm_15min[train_size:]    
    
    # ----------------- RUN MODEL  -----------------
    trace = fit_and_return_trace(cgm_train, bolus_train, basal_norm_15min, train_size, extended_size)

    # ------------ CALCULATE USEFUL VARIABLES  -----------
    plot_dict_glucose = get_plotting_variables(trace)
    
    # ------------ EVALUATE AND STORE  -----------    
    print(plot_dict_glucose['median'][train_size:])
    print(cgm_test.values)
        
    glucose_median_pred[i].append(plot_dict_glucose['median'][train_size:])
    glucose_low_50_pred[i].append(np.median(plot_dict_glucose['low_50'], axis=0)[train_size:])
    glucose_high_50_pred[i].append(np.median(plot_dict_glucose['high_50'], axis=0)[train_size:])
    glucose_low_75_pred[i].append(np.median(plot_dict_glucose['low_75'], axis=0)[train_size:])
    glucose_high_75_pred[i].append(np.median(plot_dict_glucose['high_75'], axis=0)[train_size:])
    glucose_low_95_pred[i].append(np.median(plot_dict_glucose['low_95'], axis=0)[train_size:])
    glucose_high_95_pred[i].append(np.median(plot_dict_glucose['high_95'], axis=0)[train_size:])
    
    # ------------ INTERMEDIATE SAVE  -----------   
    # Create DataFrame from the prediction lists with train_size as the index
    df_median_pred = pd.DataFrame([plot_dict_glucose['median'][train_size:]], index=[train_size])
    df_low_50_pred = pd.DataFrame([np.median(plot_dict_glucose['low_50'], axis=0)[train_size:]], index=[train_size])
    df_high_50_pred = pd.DataFrame([np.median(plot_dict_glucose['high_50'], axis=0)[train_size:]], index=[train_size])
    df_low_75_pred = pd.DataFrame([np.median(plot_dict_glucose['low_75'], axis=0)[train_size:]], index=[train_size])
    df_high_75_pred = pd.DataFrame([np.median(plot_dict_glucose['high_75'], axis=0)[train_size:]], index=[train_size])
    df_low_95_pred = pd.DataFrame([np.median(plot_dict_glucose['low_95'], axis=0)[train_size:]], index=[train_size])
    df_high_95_pred = pd.DataFrame([np.median(plot_dict_glucose['high_95'], axis=0)[train_size:]], index=[train_size])
    
    # Save DataFrame to CSV, append if file already exists
    with open('glucose_median_pred6.csv', 'a') as f:
        df_median_pred.to_csv(f, header=False)
    with open('glucose_low_50_pred6.csv', 'a') as f:
        df_low_50_pred.to_csv(f, header=False)
    with open('glucose_high_50_pred6.csv', 'a') as f:
        df_high_50_pred.to_csv(f, header=False)
    with open('glucose_low_75_pred6.csv', 'a') as f:
        df_low_75_pred.to_csv(f, header=False)
    with open('glucose_high_75_pred6.csv', 'a') as f:
        df_high_75_pred.to_csv(f, header=False)
    with open('glucose_low_95_pred6.csv', 'a') as f:
        df_low_95_pred.to_csv(f, header=False)
    with open('glucose_high_95_pred6.csv', 'a') as f:
        df_high_95_pred.to_csv(f, header=False)

    horizon_minutes -= data_resolution
    i += 1


# Model evaluation

In [ ]:
# Observed 
horizon_minutes =  1790
horizon = int(horizon_minutes / data_resolution)  # Adjust for data resolution

# Extend the time series for the horizon
train_size = len(insulin_norm_15min) - horizon
test_size = horizon
extended_size = train_size + horizon

# Define training and testing variables
cgm_train = cgm_norm_15min[:train_size]
cgm_test = cgm_norm_15min[train_size:]
bolus_train = bolus_norm_15min[:train_size]
bolus_test = bolus_norm_15min[train_size:]   

# ---------------------------------------------------------------------------------------

observed_glucose = cgm_test.values
observations_glucose = [np.arange(i, len(observed_glucose)) for i in range(len(observed_glucose))]

print(len(predictions_glucose_median))

# Extract median, highs, and lows
predictions_glucose_median = [sublist[0] for sublist in glucose_median_pred]
predictions_glucose_low_50 = [sublist[0] for sublist in glucose_low_50_pred]
predictions_glucose_high_50 = [sublist[0] for sublist in glucose_high_50_pred]
predictions_glucose_low_75 = [sublist[0] for sublist in glucose_low_75_pred]
predictions_glucose_high_75 = [sublist[0] for sublist in glucose_high_75_pred]
predictions_glucose_low_95 = [sublist[0] for sublist in glucose_low_95_pred]
predictions_glucose_high_95 = [sublist[0] for sublist in glucose_high_95_pred]

In [ ]:
def extract_nth_element(n, list_of_arrays):
    nth_elements = []
    for arr in list_of_arrays:
        try:
            nth_elements.append(arr[n])
        except IndexError:
            nth_elements.append(None)
    return nth_elements


# Model evaluation

In [ ]:
# Importing required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.tsa.ar_model import AutoReg

# Observed 
horizon_minutes = 1790 # 3780 # 2000  # 700
horizon = int(horizon_minutes / data_resolution)  # Adjust for your data resolution

# Extend the time series for the horizon
train_size = len(insulin_norm_15min) - horizon 
test_size = horizon

extended_size = train_size + horizon
integer_index = np.arange(extended_size) * data_resolution

# Define training and testing variables
cgm_train = cgm_norm_15min[:train_size]
cgm_test = cgm_norm_15min[train_size:]
bolus_train = bolus_norm_15min[:train_size]
bolus_test = bolus_norm_15min[train_size:]

print(cgm_test[-2], cgm_test[-1])


integer_index = np.arange(len(observed_glucose))  * data_resolution

# First, get a boolean mask where True indicates a NaN value in cgm_train
nan_mask = np.isnan(cgm_train)

# Then, replace NaNs in cgm_train with corresponding net_median values
cgm_train[nan_mask] = net_median[:train_size][nan_mask]


test_data = cgm_test
start = len(cgm_train)
end = start + horizon_minutes - 1

print(start, end)

# AR Model, we do not give it a exogenous variable, since this requires future values for insulin to be able to forecast. 
# # However this is not realistic.
ar_model = AutoReg(cgm_train, lags=1, seasonal=True, trend='t', period=1440//data_resolution)
ar_model_fit = ar_model.fit()
glucose_pred_ar = ar_model_fit.predict(start=start, end=end)

In [ ]:
import numpy as np

plt.rcParams.update({'font.size': 20})

fig, ax1 = plt.subplots(figsize=(20, 7))


ax1.set_xlabel('time')
ax1.set_ylabel('CGM')
ax1.plot(cgm_train, color='green', linewidth=5, label='Observed')
ax1.plot(cgm_test, color='lightgreen', label='True', linewidth=5)
ax1.plot(glucose_pred_ar, color='blue', label='Predicted', linewidth=5)
ax1.tick_params(axis='y')
ax1.axvline(x=max(cgm_train.index), color='red', linewidth=5, label='Fit split')

# instantiate a second axes that shares the same x-axis
ax2 = ax1.twinx()  
color = 'tab:red'
ax2.set_ylabel('Count of NaNs', color=color)  
n, bins, patches = ax2.hist(nan_indices, bins=50, alpha=0.3, color='red')
ax2.tick_params(axis='y', labelcolor=color)

# Day lines
for i in range(0, len(series.index)):
    if series.index[i].hour == 0 and series.index[i].minute == 0:
        ax1.axvline(x=series.index[i], color='grey', linestyle='--')

ax1.set_xlim(min(cgm_train.index), max(cgm_test.index))
fig.tight_layout()  # otherwise the right y-label is slightly clipped
ax1.legend()
plt.show()


In [ ]:
from sklearn.metrics import mean_squared_error
from math import sqrt
import numpy as np
import matplotlib.pyplot as plt

look_ahead_intervals = [30, 60, 90, 120]  # Define the look ahead intervals

# Arrays to store predictions for each interval
predictions_arrays = [np.zeros(len(test_data)) for _ in look_ahead_intervals]

# Sliding window predictions
for t in range(len(test_data)):
    # Updating the train_data with the latest known value
    train_data = cgm_norm_15min[:train_size+t]
    train_data = train_data.fillna(0)
    
    for i, interval in enumerate(look_ahead_intervals):
        # Define the end point for the interval
        end_point = interval // data_resolution  # Convert minutes to steps
        
        # If there's not enough future data to predict, break the loop
        if t + end_point >= len(test_data):
            break
        
        # Define prediction horizon
        start = len(train_data)
        end = start + end_point - 1

        # AR Model
        arx_model = AutoReg(train_data, lags=1, seasonal=True, trend='t', period=1440//data_resolution)
        arx_model_fit = arx_model.fit()
        glucose_pred_arx = arx_model_fit.predict(start=start, end=end)
        
        # Store the prediction
        predictions_arrays[i][t + end_point] = glucose_pred_arx.iloc[-1]
        
    # Print progress
    if t % 10 == 0:
        print('Predicted up to point:', t)

In [ ]:

# Plot the results
plt.figure(figsize=(14,7))
plt.plot(test_data.index, test_data.values, label='Test Data', linewidth=5)

for i, interval in enumerate(look_ahead_intervals):
    plt.plot(test_data.index, predictions_arrays[i], label='Predictions {} min ahead'.format(interval))

plt.title('Blood Glucose Level Predictions')
plt.xlabel('Time')
plt.ylabel('Normalized Glucose Level')
plt.legend()
plt.show()

# RMSE calculations
for i, interval in enumerate(look_ahead_intervals):
    actual_values = test_data[:len(predictions_arrays[i])]
    predicted_values = predictions_arrays[i][:len(test_data)]
    
    mse = mean_squared_error(actual_values, predicted_values)
    rmse = sqrt(mse)
    print('RMSE for the look ahead of {} minutes: {}'.format(interval, round(rmse * 18.0182, 2)))

# Final plot

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Set style and color palette
sns.set_style("whitegrid")
sns.set_palette("husl")

integer_index = np.arange(len(observed_glucose))  * data_resolution

# Create the plot
plt.figure(figsize=(15,8))  # Optional: set the figure size

# Plot the observed glucose
plt.plot(integer_index, observed_glucose, label='Observed Glucose', color='green', alpha=1, linewidth=8, zorder=10)

# Plot our predictions
plt.plot(predicted_range * data_resolution, predicted, linewidth=5, color='black', linestyle='-', label='Our Predictions')

# Plot AR model
plt.plot(integer_index[4:], predictions_arrays[3][4:], color='blue', linewidth=5, alpha=0.6, label='AR Predictions')

# Plot the prediction intervals
plt.fill_between(predicted_range * data_resolution, predicted_low_50, predicted_high_50, color='red', alpha=0.6)
plt.fill_between(predicted_range * data_resolution, predicted_low_75, predicted_high_75, color='red', alpha=0.4)
plt.fill_between(predicted_range * data_resolution, predicted_low_95, predicted_high_95, color='red', alpha=0.2)

# Lines and patches
plt.fill_between(integer_index, 3, 10, color='lightgreen', alpha=0.3, zorder=-10)

# Format the plot
plt.title(f'Glucose Predictions ({ph_minutes} min PH)', fontsize=25)


plt.axvline(x=350, color='black', linestyle='--', zorder=10, linewidth=2)
plt.axvline(x=1790, color='black', linestyle='--', zorder=10, linewidth=2)


plt.xlabel('Time', fontsize=18)
plt.ylabel('mmol/L', fontsize=18)
plt.ylim(2.5, 12.5)

plt.legend(fontsize=15)
plt.tight_layout()
plt.show()


# Getting the saved results

In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np

# Median
predictions_glucose_median_df = pd.read_csv('glucose_median_pred6.csv', header=None, index_col=0)
glucose_median_pred = [row.dropna().tolist() for index, row in predictions_glucose_median_df.iterrows()]

# Upper 95
predictions_glucose_median_df = pd.read_csv('glucose_high_95_pred6.csv', header=None, index_col=0)
glucose_high_95_pred = [row.dropna().tolist() for index, row in predictions_glucose_median_df.iterrows()]

# Upper 75
predictions_glucose_median_df = pd.read_csv('glucose_high_75_pred6.csv', header=None, index_col=0)
glucose_high_75_pred = [row.dropna().tolist() for index, row in predictions_glucose_median_df.iterrows()]

# Upper 50
predictions_glucose_median_df = pd.read_csv('glucose_high_50_pred6.csv', header=None, index_col=0)
glucose_high_50_pred = [row.dropna().tolist() for index, row in predictions_glucose_median_df.iterrows()]

# Lower 95
predictions_glucose_median_df = pd.read_csv('glucose_low_95_pred6.csv', header=None, index_col=0)
glucose_low_95_pred = [row.dropna().tolist() for index, row in predictions_glucose_median_df.iterrows()]

# Lower 75
predictions_glucose_median_df = pd.read_csv('glucose_low_75_pred6.csv', header=None, index_col=0)
glucose_low_75_pred = [row.dropna().tolist() for index, row in predictions_glucose_median_df.iterrows()]

# Lower 50
predictions_glucose_median_df = pd.read_csv('glucose_low_50_pred6.csv', header=None, index_col=0)
glucose_low_50_pred = [row.dropna().tolist() for index, row in predictions_glucose_median_df.iterrows()]




In [ ]:
observed_glucose = cgm_test.values
observations_glucose = [np.arange(i, len(observed_glucose)) for i in range(len(observed_glucose))]

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Set style and color palette
sns.set_style("whitegrid")
sns.set_palette("husl")

# Create the plot
plt.figure(figsize=(10,6))  # Optional: set the figure size

integer_index = np.arange(len(observed_glucose))  * data_resolution

# Plot the observed glucose and predicted glucose
plt.plot(integer_index, observed_glucose, label='Observed Glucose', color='green', alpha=1, linewidth=3, marker='x')

# Plot the nth prediction
ph_minutes = 120
ph = (ph_minutes // data_resolution) - 1
predicted = extract_nth_element(ph, predictions_glucose_median)[:len(integer_index) - ph]
predicted_range = observations_glucose[ph]

observed = observed_glucose[ph:]

plt.plot(predicted_range * data_resolution, predicted, linewidth=2, color='red', linestyle='-', label=f'{ph_minutes} min PH', marker='x')

# Extract values for 50%, 75% and 95% prediction intervals
predicted_low_50 = extract_nth_element(ph, predictions_glucose_low_50)[:len(integer_index) - ph]
predicted_high_50 = extract_nth_element(ph, predictions_glucose_high_50)[:len(integer_index) - ph]
predicted_low_75 = extract_nth_element(ph, predictions_glucose_low_75)[:len(integer_index) - ph][:len(integer_index) - ph]
predicted_high_75 = extract_nth_element(ph, predictions_glucose_high_75)[:len(integer_index) - ph]
predicted_low_95 = extract_nth_element(ph, predictions_glucose_low_95)[:len(integer_index) - ph]
predicted_high_95 = extract_nth_element(ph, predictions_glucose_high_95)[:len(integer_index) - ph]

# Plot the prediction intervals
plt.fill_between(predicted_range * data_resolution, predicted_low_50, predicted_high_50, color='red', alpha=0.1, label='50% Prediction Interval')
plt.fill_between(predicted_range * data_resolution, predicted_low_75, predicted_high_75, color='red', alpha=0.2, label='75% Prediction Interval')
plt.fill_between(predicted_range * data_resolution, predicted_low_95, predicted_high_95, color='red', alpha=0.3, label='95% Prediction Interval')


# Lines and patches
plt.fill_between(integer_index, 3, 10, color='lightgreen', alpha=0.2)

# Format the plot
plt.title('Glucose level over time with Predictions', fontsize=16)

plt.xlabel('Time', fontsize=14)
plt.ylabel('mmol/L', fontsize=14)
plt.ylim(2, 13)
# plt.xlim(0, 1440)

plt.legend(fontsize=12)
plt.tight_layout()
plt.show()


# Clarke error grid

In [ ]:



import matplotlib.pyplot as plt
import matplotlib.patches as patches


def clarke_error_grid(ref_values, pred_values, title_string):

    #Checking to see if the lengths of the reference and prediction arrays are the same
    assert (len(ref_values) == len(pred_values)), "Unequal number of values (reference : {}) (prediction : {}).".format(len(ref_values), len(pred_values))

    #Checks to see if the values are within the normal physiological range, otherwise it gives a warning
    if max(ref_values) > 400 or max(pred_values) > 400:
        print("Input Warning: the maximum reference value {} or the maximum prediction value {} exceeds the normal physiological range of glucose (<400 mg/dl).").format(max(ref_values), max(pred_values))
    if min(ref_values) < 0 or min(pred_values) < 0:
        print("Input Warning: the minimum reference value {} or the minimum prediction value {} is less than 0 mg/dl.").format(min(ref_values),  min(pred_values))

    #Clear plot
    plt.clf()

    #Set up plot
    plt.scatter(ref_values, pred_values, marker='o', color='darkblue', s=15, alpha=1)
    plt.title(title_string + " Clarke Error Grid")
    plt.xlabel("Reference Concentration (mg/dl)")
    plt.ylabel("Prediction Concentration (mg/dl)")
    plt.xticks([0, 50, 100, 150, 200, 250, 300, 350, 400])
    plt.yticks([0, 50, 100, 150, 200, 250, 300, 350, 400])
    plt.gca().set_facecolor('white')

    #Set axes lengths
    plt.gca().set_xlim([0, 400])
    plt.gca().set_ylim([0, 400])
    plt.gca().set_aspect((400)/(400))

    #Plot zone lines
    plt.plot([0,400], [0,400], ':', c='black')                      #Theoretical 45 regression line
    plt.plot([0, 175/3], [70, 70], '-', c='black')
    #plt.plot([175/3, 320], [70, 400], '-', c='black')
    plt.plot([175/3, 400/1.2], [70, 400], '-', c='black')           #Replace 320 with 400/1.2 because 100*(400 - 400/1.2)/(400/1.2) =  20% error
    plt.plot([70, 70], [84, 400],'-', c='black')
    plt.plot([0, 70], [180, 180], '-', c='black')
    plt.plot([70, 290],[180, 400],'-', c='black')
    # plt.plot([70, 70], [0, 175/3], '-', c='black')
    plt.plot([70, 70], [0, 56], '-', c='black')                     #Replace 175.3 with 56 because 100*abs(56-70)/70) = 20% error
    # plt.plot([70, 400],[175/3, 320],'-', c='black')
    plt.plot([70, 400], [56, 320],'-', c='black')
    plt.plot([180, 180], [0, 70], '-', c='black')
    plt.plot([180, 400], [70, 70], '-', c='black')
    plt.plot([240, 240], [70, 180],'-', c='black')
    plt.plot([240, 400], [180, 180], '-', c='black')
    plt.plot([130, 180], [0, 70], '-', c='black')

    #Add zone titles
    plt.text(30, 15, "A", fontsize=15)
    plt.text(370, 260, "B", fontsize=15)
    plt.text(280, 370, "B", fontsize=15)
    plt.text(160, 370, "C", fontsize=15)
    plt.text(160, 15, "C", fontsize=15)
    plt.text(30, 140, "D", fontsize=15)
    plt.text(370, 120, "D", fontsize=15)
    plt.text(30, 370, "E", fontsize=15)
    plt.text(370, 15, "E", fontsize=15)

    #Statistics from the data
    zone = [0] * 5
    for i in range(len(ref_values)):
        if (ref_values[i] <= 70 and pred_values[i] <= 70) or (pred_values[i] <= 1.2*ref_values[i] and pred_values[i] >= 0.8*ref_values[i]):
            zone[0] += 1    #Zone A

        elif (ref_values[i] >= 180 and pred_values[i] <= 70) or (ref_values[i] <= 70 and pred_values[i] >= 180):
            zone[4] += 1    #Zone E

        elif ((ref_values[i] >= 70 and ref_values[i] <= 290) and pred_values[i] >= ref_values[i] + 110) or ((ref_values[i] >= 130 and ref_values[i] <= 180) and (pred_values[i] <= (7/5)*ref_values[i] - 182)):
            zone[2] += 1    #Zone C
        elif (ref_values[i] >= 240 and (pred_values[i] >= 70 and pred_values[i] <= 180)) or (ref_values[i] <= 175/3 and pred_values[i] <= 180 and pred_values[i] >= 70) or ((ref_values[i] >= 175/3 and ref_values[i] <= 70) and pred_values[i] >= (6/5)*ref_values[i]):
            zone[3] += 1    #Zone D
        else:
            zone[1] += 1    #Zone B

    return plt, zone

plt.figure(figsize=(10, 7))
pltt, zone = clarke_error_grid(np.array(observed) * 18.0182, np.array(predicted) * 18.0182, "Ours 120 min -")
total_points = sum(zone)
percentage_zone = [round(z/total_points*100, 2) for z in zone]

print(percentage_zone)
plt.show()

plt.figure(figsize=(10, 7))
pltt, zone = clarke_error_grid(np.array(observed[1:]) * 18.0182, np.array(predictions_arrays[3][4:]) * 18.0182, "AR 120 min -")
total_points = sum(zone)
percentage_zone = [round(z/total_points*100, 2) for z in zone]
print(percentage_zone)

plt.show()

